In [ ]:
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 28.0 MB/s 
     |████████████████████████████████| 831.4 MB 2.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.0
    Uninstalling torchtext-0.13.0:
      Successfully uninstalled torchtext-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.


In [ ]:
!pip install pytorch-crf
!pip install deeppavlov
!pip install transformers


import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

!pip install razdel

import razdel
import random
import math
import time

from torchtext.legacy.data import Field

import torch.nn.functional as F

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)



cuda


In [ ]:

from razdel import tokenize
import re


with open('data1.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    lines = [re.sub("[`~*)@#%(&$_^]’", '', line) for line in lines]
    lines = [re.sub("\d+", 'number', line) for line in lines]
    lines = [line.replace('«','') for line in lines]
    lines = [line.replace('»','') for line in lines]
    lines = [line.replace('#','') for line in lines]
    lines = [line.replace('@','') for line in lines]
    lines = [line.replace('_','') for line in lines]
    lines = [line.replace('\\','') for line in lines]
    lines = [line.replace('&','') for line in lines]
    lines = [line.replace('—','') for line in lines]
    lines = [line.replace('–','') for line in lines]
    lines = [line.replace('-',' ') for line in lines]
    lines = [line.replace('…','.') for line in lines]
    lines = [line.replace(';',',') for line in lines]
    lines = [line.replace(')','') for line in lines]
    lines = [line.replace('(','') for line in lines]
    lines = [line.replace('?','.') for line in lines]
    lines = [line.replace('!','.') for line in lines]
    lines = [line.replace(':',',') for line in lines]
    lines = [line.replace('„',',') for line in lines] 
    lines = [line.replace('*','') for line in lines] 
    lines = [line.replace("'",'') for line in lines] 
    lines = [line.replace('"','') for line in lines] 
    lines = [line.replace('...','.') for line in lines] 
    lines = [line.replace('..','') for line in lines] 
    lines = [line.strip() for line in lines]
    lines_rus = [line for line in lines if line]

    print(lines_rus[:10])
    

lines = lines_rus
max_len = 80
corpus = []

for n, i in enumerate(lines):
    
    string = [_.text for _ in list(tokenize(i))]
    if 5 <= len(string) <= max_len:
        corpus.append(string)
    else:
        pass


    if (n%1000) == 0:
        print(n)

test = corpus[1800:]
corpus = corpus[:1800]

print(corpus[0])
print(corpus[1])

['Finally tonight, its prom season / and Steve Hartman met the queen and her king, / On the Road.', 'At Mountain Heritage High School in Burnsville, North Carolina, / Rachel Newberry caught Ben Robinson / completely off guard.', 'You didnt know / what was happening, did you.', 'Did you tell you were a little surprised, / you were a little shocked.', 'Yes, yes.', 'Ben, who has Down syndrome, had no idea / Rachel was going to ask him / to senior prom.', 'Every year we see stories like this   / a typical kid inviting / a special needs kid to prom.', 'Stories of kindness.', 'But what makes this so different / is that Rachel wasnt trying to be kind.', 'In fact, when she posted the video / and strangers started commenting / that she was this amazing, incredible young lady / with a very good heart, / Rachel was a little put off.']
0
1000
2000
['Finally', 'tonight', ',', 'its', 'prom', 'season', '/', 'and', 'Steve', 'Hartman', 'met', 'the', 'queen', 'and', 'her', 'king', ',', '/', 'On', 'the',

In [ ]:
from sklearn.model_selection import train_test_split
import string

print(len(corpus))
print(len(test))

TRG = Field(init_token = '<sos>', 
            eos_token = '<eos>')

1800
311


In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
import transformers
from transformers import BertTokenizer
from transformers import BertModel


punct = [',','.','?','/']




X = [[c for c in s if c not in punct] for s in corpus]

Y = []
dir = 'bert-large-cased'
tokenizer = BertTokenizer.from_pretrained(dir, do_lower_case = False, max_length = 100, model_max_length = 100)

corpus = [' '.join(i) for i in corpus]
for snt in corpus:
    
    src = tokenizer([snt], return_tensors="pt", padding='max_length')

    res = []
    p = {128:',', 132:'.'}
    r = src['input_ids'][0]
    for i in range(len(r)-1):

        if r[i].item() not in p.keys():
            if r[i+1].item() in p:
                res.append(p[r[i+1].item()])
            else:
                res.append('_')
    Y.append(res)



n = 23
ans1 = X[n]
ans = Y[n]
print('SRC:', ans1)
print('TRG:', ans)



print(X[:10])
X = [' '.join(i) for i in X]
print(X[:10])

train_src, valid_src, train_trg, valid_trg = train_test_split(X, Y, test_size=0.2, shuffle = True, random_state=seed)

l = [train_src, valid_src, train_trg, valid_trg]
for i in l:
  print(len(i))



TRG.build_vocab([i for i in Y], min_freq = 10)
print("Number of tokens in vocabulary", len(TRG.vocab))
print(TRG.vocab.stoi.items())

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

SRC: ['Steve', 'Hartman', 'On', 'the', 'Road', 'in', 'Burnsville', 'North', 'Carolina']
TRG: ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_']
[['Finally', 'tonight', 'its', 'prom', 'season', 'and', 'Steve', 'Hartman', 'met', 'the', 'queen', 'and', 'her', 'king', 'On', 'the', 'Road'], ['At', 'Mountain', 'Heritage', 'High', 'School', 'in', 'Burnsville', 'North', 'Carolina', 'Rachel', 'Newberry', 'caught', 'Ben', 'Robinson', 'completely', 'off', 'guard'], ['You', 'didnt', 'know', 'what', 'was', 'happening', 'did', 'you'], ['Did', 'you', 'tell', 'you'

In [ ]:
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pack_sequence
from torch.nn.utils.rnn import PackedSequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pad_sequence


class PunctDataset(Dataset):
    def __init__(self, src_data, trg_data):

        self.src = src_data
        self.trg = trg_data

        print(self.src[0])
        print(self.trg[0])
        
    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        
        return self.src[idx], [TRG.vocab.stoi[i] for i in self.trg[idx]] + [TRG.vocab.stoi['<eos>']]
        
    def collate_fn(self, batch):
        
        return [pair[0] for pair in batch], pad_packed_sequence(pack_sequence([torch.tensor(pair[1]) for pair in batch], enforce_sorted=False), padding_value = TRG.vocab.stoi['<pad>'], total_length = 100, batch_first = True)[0]

In [ ]:
from torchcrf import CRF

class PunctCorrector(nn.Module):
    def __init__(self, num_cls):
        """
        :param: num_cls is the size of target space

        """
        super().__init__()
        
        self.num_cls = num_cls

        self.ln = nn.Linear(1024, 256)
        self.ln2 = nn.Linear(256, num_cls)
        
        self.model_CRF = CRF(num_cls, batch_first = True)

        torch.nn.init.constant_(self.model_CRF.start_transitions[3], -1e9) # to prevent EOS-tag transitions
        torch.nn.init.constant_(self.model_CRF.transitions[3], -1e9)


    def forward(self, embedded, trg = None, mask = None, mode = 'trn'):
        """
        :param: src sentences
        :param: trg sentences
        :param: mask of trg for CFR
        :param: mode - train or eval

        """

        outputs = self.ln2(F.relu(self.ln(embedded)))

        if mode == 'trn':

            if mask != None:
                loss = -self.model_CRF(outputs, trg, mask=mask)
            else:
                loss = -self.model_CRF(outputs, trg)
            out = self.model_CRF.decode(outputs)

            return loss, out

        if mode == 'eval':
             out = self.model_CRF.decode(outputs)

             return out

In [ ]:
number_classes = len(TRG.vocab)
batch_size_trn = 64
batch_size_eval = 256



dataset_trn = PunctDataset(train_src, train_trg)
dataloader_trn = DataLoader(dataset_trn, 
                              batch_size=batch_size_trn,
                              shuffle=True,
                              collate_fn=dataset_trn.collate_fn)

dataset_val = PunctDataset(valid_src, valid_trg)
dataloader_val = DataLoader(dataset_val, 
                              batch_size=batch_size_eval,
                              shuffle=True,
                              collate_fn=dataset_val.collate_fn)



model = PunctCorrector(number_classes).to(device)

dir = 'bert-large-cased'
tokenizer = BertTokenizer.from_pretrained(dir, do_lower_case = False, max_length = 256, model_max_length = 100)
roberta = BertModel.from_pretrained(dir).to(device)

We decided you know lets see if we can buy the car
['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_']
His mission to find her ring began with a thorough screening of the security footage
['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(model)

total_params = sum(p.numel() for p in model.parameters()if p.requires_grad)
print('Number of trainable parameters: ', total_params)
print('\n')

PunctCorrector(
  (ln): Linear(in_features=1024, out_features=256, bias=True)
  (ln2): Linear(in_features=256, out_features=5, bias=True)
  (model_CRF): CRF(num_tags=5)
)
Number of trainable parameters:  263720




In [ ]:
def train(tokenizer, roberta, model, iterator, optimizer):
    
    model.train()
    roberta.eval()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        model.train()
        
        src = batch[0]
        trg = batch[1].to(device)

        mask = trg.clone().detach()
        mask[mask==1] = 0
        mask[mask!=0] = 1
        mask = torch.tensor(mask, dtype=torch.uint8).to(device)

        
        optimizer.zero_grad()

        src = tokenizer(src, return_tensors="pt", padding='max_length')

        with torch.no_grad():
            embedded = roberta.forward(**src.to(device))[0]

        loss, output = model(embedded, trg, mask)

        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(tokenizer, roberta, model, iterator):
    
    model.eval()
    roberta.eval()

    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0]
            trg = batch[1].to(device)

            mask = trg.clone().detach()
            mask[mask==1] = 0
            mask[mask!=0] = 1
            mask = torch.tensor(mask, dtype=torch.uint8).to(device)

            src = tokenizer(src, return_tensors="pt", padding='max_length')
            
            embedded = roberta.forward(**src.to(device))[0]   

            loss, output = model(embedded, trg, mask)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
max_epochs = 25
lr = 1e-2
weight_decay = 5e-6
step_size = 3
gamma = 0.9

message = 'Training starts... ' + '\nNumber trn batches: '+str(len(dataloader_trn)) + '\nNumber eval batches: '+str(len(dataloader_val))


optimizer = optim.Adam(model.parameters(), lr = lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)




print(message)
best_valid_loss = float('inf')
for epoch in range(1, max_epochs+1):

    train_loss = round(train(tokenizer, roberta, model, dataloader_trn, optimizer), 5)
    valid_loss = round(evaluate(tokenizer, roberta, model, dataloader_val), 5)
    scheduler.step()
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
        print("Saving best model's weights")
        
    

    print(f"| EPOCH №{epoch:3.0f} | TRN LOSS {train_loss:.5f} | VAL LOSS {valid_loss:.5f} | LR {scheduler.get_last_lr()[0]:.7f}")

Training starts... 
Number trn batches: 23
Number eval batches: 2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Saving best model's weights
| EPOCH №  1 | TRN LOSS 385.35041 | VAL LOSS 0.00000 | LR 0.0100000
| EPOCH №  2 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0100000
| EPOCH №  3 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0090000
| EPOCH №  4 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0090000
| EPOCH №  5 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0090000
| EPOCH №  6 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0081000
| EPOCH №  7 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0081000
| EPOCH №  8 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0081000
| EPOCH №  9 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0072900
| EPOCH № 10 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0072900
| EPOCH № 11 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0072900
| EPOCH № 12 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0065610
| EPOCH № 13 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0065610
| EPOCH № 14 | TRN LOSS 0.00000 | VAL LOSS 0.00000 | LR 0.0065610
| EPOCH № 15 | TRN LOSS 0.00000 | VAL LOSS 0.0

In [ ]:
import string
from nltk.tokenize import wordpunct_tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize



def check_one(reference, hypothesis):
    correct = 0
    incorrect = 0
    ref = wordpunct_tokenize(reference)
    hyp = wordpunct_tokenize(hypothesis)
    ref_i, hyp_i = 0, 0
    punct_places = 1e-7
    while ref_i < len(ref) and hyp_i < len(hyp):
        need_punct_check_ref = False
        need_punct_check_hyp = False
        cur_ref = ref[ref_i]
        if cur_ref in string.punctuation:
            need_punct_check_ref = True
            punct_places += 1
        cur_hyp = hyp[hyp_i]
        if cur_hyp in string.punctuation:
            need_punct_check_hyp = True
        if need_punct_check_ref and need_punct_check_hyp:
            if cur_ref == cur_hyp:
                correct += 1
            else:
                incorrect += 1
            ref_i += 1
            hyp_i += 1
            continue

        if need_punct_check_ref and not need_punct_check_hyp:
            incorrect += 1
            ref_i += 1
            continue

        if not need_punct_check_ref and need_punct_check_hyp:
            incorrect += 1
            hyp_i += 1
            continue

        assert cur_hyp == cur_ref, "The phrases are inconsistent!"
        ref_i += 1
        hyp_i += 1

    if punct_places == 1e-7:
        if correct == 0:
            if incorrect == 0:
                return 1
            else:
                return correct/punct_places - incorrect/(2 * len(reference))
        else:
            return correct/punct_places - incorrect/(2 * len(reference))
    else:
         return correct/punct_places - incorrect/(2 * len(reference))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:





model.load_state_dict(torch.load('model.pt'))




data = test
if not isinstance(data[0], str):
    data = [' '.join(i) for i in data]

sentences = data
#sentences = ['''Уникальный макет очень точно копирует центральную часть города. В нем воссозданы мозаики, барельефы, дорожная разметка, подземные переходы и даже пандусы.''']


q = 0
for n, sentence_gold in enumerate(sentences):

    sentence = " ".join(filter(lambda x: x not in string.punctuation, word_tokenize(sentence_gold)))

    sentence_token = tokenizer(sentence, return_tensors="pt", padding='max_length')
    sentence_token = tokenizer.convert_ids_to_tokens(sentence_token['input_ids'][0])
    sentence_token = [i for i in sentence_token if i != '[PAD]']
   

    src = tokenizer(sentence, return_tensors="pt", padding='max_length')
    embedded = roberta.forward(**src.to(device))[0]   
    output = model(embedded, mode = 'eval')

    ans = []
    for t in output[0]:
            ans.append(TRG.vocab.itos[t])

    
    res = ''
    if ans[0] == '_':
        pass
    else:
        res += ans[0]
        res += ' '

    for i in range(1, len(sentence_token)-1):

                if ans[i] == '_':
                    if '##' in sentence_token[i]:
                        res += sentence_token[i].replace("#", "")
                        
                    else:
                        res += ' '
                        res += sentence_token[i]                                       
                        
                else: 
                    if '##' in sentence_token[i]:
                        res += sentence_token[i].replace("#", "")
                        if '##' not in sentence_token[i+1]:
                            res += ans[i]
                        else:
                            pass
                    else:    
                        res += ' '          
                        res += sentence_token[i]
                        if '##' not in sentence_token[i+1]:
                            res += ans[i]
                        else:
                            pass
                        
    

    q += check_one(sentence_gold, res) * 100

    print(sentence_gold)
    #print(sentence_token)
    #print(ans)
    print(res)
    print(check_one(sentence_gold, res) * 100)
    

print("Quality: {:.2f}%".format(q / n+1))
print("Number of sentences in quality calcs: {}.".format(n+1))

This week , we tell about / one of the most successful American businesswomen .
 This week we tell about one of the most successful American businesswomen
-1.2658227848101267
Mary Kathlyn Wagner was born / in the state of Texas in number .
 Mary Kathlyn Wagner was born in the state of Texas in number
-0.78125
For much of her childhood , / she cared for her sick father / while her mother worked long hours / at a public eating place .
 For much of her childhood she cared for her sick father while her mother worked long hours at a public eating place
-1.6
Mary Kay married Ben Rogers / when she was seventeen years old .
 Mary Kay married Ben Rogers when she was seventeen years old
-0.78125
They had three children / before he left home / to serve in World War ‡U .
 They had three children before he left home to serve in World War ‡ U
-1.3513513513513513
When he returned , their marriage ended .
 When he returned their marriage ended
-1.2195121951219512
Mary Kay looked for a job / so she cou

AssertionError: ignored